<a class="anchor" id="toc-go-back"></a>
# Assignment 3: Day-ahead scheduling from the perspective of the system operator

### Table of Contents
* [1. Loading the data](#data-loading)
* [2. Setup optimization problem](#optimziation-problem-setup)
* [3. Running the optimization model ](#run-optimization)
* [4. Can we solve the problem using machine learning?](#machine-learning)
* [5. Extra ](#extra)

First of all, we refer to the hand-in for an in-depth description of the problem formulation and the optimization problem. This notebooks focuses on implementing the model using ```Gurobipy``` and an additional part using machine learning. 

We start by loading the relevant modules.

In [5]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import gurobipy as grb

In [6]:
DATA_DIR = Path('../data')

<a class="anchor" id="data-loading"></a>
## 1. Loading the data

We initially load the data files given to us with the assignment description. This will later on be used as input to the optimization model from which we can then find the optimal values of the decision variables.

In [11]:
# Load data files from csv-files
pmax    = pd.read_csv(DATA_DIR / 'pgmax.csv')
pmin    = pd.read_csv(DATA_DIR / 'pgmin.csv')
ru      = pd.read_csv(DATA_DIR / 'ramp.csv')
UT      = pd.read_csv(DATA_DIR / 'lu.csv')
DT      = pd.read_csv(DATA_DIR / 'ld.csv')    
demand  = pd.read_csv(DATA_DIR / 'Demand.csv', sep=';')   
c_op    = pd.read_csv(DATA_DIR / 'cost_op.csv') 
c_st    = pd.read_csv(DATA_DIR / 'cost_st.csv') 
PTDF    = pd.read_csv(DATA_DIR / 'PTDF.csv', sep=';') 
busgen  = pd.read_csv(DATA_DIR / 'busgen.csv', sep=';')
busload = pd.read_csv(DATA_DIR / 'busload.csv', sep=';')
fmax    = pd.read_csv(DATA_DIR / 'fmax.csv')

# ???????????????????????????????????????????????????????????????????????????
Hg      = np.dot(PTDF, busgen)
Hl      = np.dot(PTDF, busload)

With the data set loaded, we can now define the basics underlying the optimization problem - thus, we start by defining the numerical values of the parameters.

In [ ]:
N_g     = busgen.shape[1]   # the number of generator units
N_t     = demand.shape[0]   # next 24 hours
N_load  = busload.shape[1]  # the number of load buses
N_lines = PTDF.shape[0]     # the number of transmission lines

<a class="anchor" id="optimization-problem-setup"></a>
## 2. Setup optimization problem

In [12]:
model = grb.Model()

Restricted license - for non-production use only - expires 2024-10-28


Now, we are ready to define the decision variables of the optimization problem. These relate to the start-up status and production of generator units at each time step of the day. We additionally include a variable for denoting the on/off status. As we are given a data file for start-up costs for each generation unit, we approach the problem with the start-up cost being time-independent and consider it as an input rather than a decision variable.

In [26]:
# Add variables for each generator unit at each time step for specifying on/off and start-up status
b   = model.addVars(N_g, N_t, vtype=grb.GRB.BINARY)
u   = model.addVars(N_g, N_t, vtype=grb.GRB.BINARY)

# Add variable denoting the power output of each generator unit at each time step
p   = model.addVars(N_g, N_t, vtype=grb.GRB.CONTINUOUS)

# We add slack variables for ensuring feasability of the power balance equations
eps     = model.addVars(N_t, 1, vtype=grb.GRB.CONTINUOUS)
delta   = model.addVars(N_t, 1, vtype=grb.GRB.CONTINUOUS)

# Make variable updates effective
model.update()

Next, we add the constraints to the optimization model. Overall these concern:
1) Power balance equation, i.e. the total production must equal the total demand. Here, we add the concept of slack variable for feasibility - i.e. making sure that the problem can be solved even if in some cases the demand is not met.
2) Generation limits, ensuring that the generator units do not produce more than their maximum capacity.
3) Line technical limits, ensuring that the flow between busses does not exceed the maximum capacity of the line.
4) Start-up constraints for turning on a generator unit
5) Ramp-up and ramp-down constraints, ensuring...
6) Binary constraints for generator unit status (on/off).

In [53]:
for t in range(N_t):

    ### 1) POWER BALANCE EQUATION ###
    # Add power balance constraints for each time step
    model.addConstr(sum(p[j, t] for j in range(N_g)) == sum(demand.iloc[t, i] for i in range(N_load)) + eps[t, 0] - delta[t, 0])
    # Add positive constraints for the slack variables
    model.addConstr(eps[t, 0] >= 0)
    model.addConstr(delta[t, 0] >= 0)

    ### 2) GENERATION LIMITS ###
    ...

    ### 3) LINE FLOW LIMITS ###
    ...

    ### 4) START-UP COSTS ###
    ...

    ### 5) RAMPING CONSTRAINTS ###
    ...

    ### 6) BINARIZE GENERATOR ON/OFF AND START-UP STATUS ###
    ...
    
model.update()

Lastly, we define the objective function for the optimization problem. This is defined as minimizing the total cost - i.e. production cost and start-up cost. In this case, we assume deterministic start-up costs for each generator unit - in other words, we assume that they are time-independent.

In [55]:
# s_g = u * c_st

<a class="anchor" id="run-optimization"></a>
## 3. Running the optimization model

<a class="anchor" id="machine-learning"></a>
## 4. Can we solve the problem using machine learning?

<a class="anchor" id="extra"></a>
## 5. Extra